In [50]:
import numpy as np
import gurobipy as gp
from gurobipy import GRB

from sklearn.datasets import load_diabetes

In [73]:
np.random.seed(2025)
n = 10
# X = np.random.randn(1000,n)
data = load_diabetes()
X = data.data
X = (X - np.mean(X,axis=0))/np.std(X, axis=0)
Sigma = np.cov(X.T)
inv_sig = np.linalg.inv(Sigma)

In [74]:
# Create model
model = gp.Model("log_trace_optimization")

# Add variables
Gamma = model.addMVar((n, n), lb=-GRB.INFINITY, ub=10, name="Gamma")  # Ensure Gamma_ii > 0 for log()
Gamma_diag = model.addVars(n, lb=0, ub=10)

model.addConstrs(Gamma[i, i] == Gamma_diag[i] for i in range(n))

# Add auxiliary variables for -log(Gamma_ii)
log_terms = model.addVars(n, lb=-GRB.INFINITY, name="LogTerms")

# Define piecewise-linear approximation for -log(Gamma_ii)
for i in range(n):
    model.addGenConstrLog(Gamma_diag[i], log_terms[i])  # Gurobi's log function



# Define objective
# Quadratic trace term: tr(Gamma * Gamma.T * Sigma)
# quad_term = gp.quicksum(Gamma[i, j] * Gamma[i, j] * Sigma[j, j] for i in range(n) for j in range(n))
product_term = Gamma@Gamma.T@Sigma
quad_term = gp.quicksum(product_term[i, i] for i in range(n))
log_sum = gp.quicksum(-2 * log_terms[i] for i in range(n))

# Set the objective
model.setObjective(log_sum + quad_term, GRB.MINIMIZE)

# Solve
model.optimize()

# Retrieve results
if model.status == GRB.OPTIMAL:
    Gamma_opt = np.array([[Gamma[i, j].X for j in range(n)] for i in range(n)])
    print("Optimal Gamma:")
    print(np.round(Gamma_opt,2))
else:
    print("Optimization did not converge.")


Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (mac64[arm])

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 10 rows, 120 columns and 20 nonzeros
Model fingerprint: 0x6f270b24
Model has 550 quadratic objective terms
Model has 10 general constraints
Variable types: 120 continuous, 0 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+00, 2e+00]
  QObjective range [1e-01, 4e+00]
  Bounds range     [1e+01, 1e+01]
  RHS range        [0e+00, 0e+00]
Presolve added 9 rows and 100 columns
Presolve time: 0.05s
Presolved: 19 rows, 220 columns, 3317 nonzeros
Presolved model has 55 quadratic objective terms
Variable types: 212 continuous, 8 integer (1 binary)
Found heuristic solution: objective -1.1962945

Root relaxation: objective -6.582006e+00, 142 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl

In [75]:
-np.log(np.linalg.det(inv_sig)) + np.trace(inv_sig@Sigma)

2.2729915753251726

In [76]:
np.trace(Gamma_opt@Gamma_opt.T@Sigma)

9.983192306675578

In [80]:
[np.log(Gamma_diag[i].x) for i in range(n)]

[0.09531017988523396,
 0.1254685239482601,
 0.20666998669731618,
 0.18232155686728796,
 2.040734178548454,
 1.83258146375431,
 1.3648896656644154,
 1.0925524889348406,
 1.1526011247463321,
 0.19009360892052415]

In [81]:
[log_terms[i].x for i in range(n)]

[0.09531017988523396,
 0.1252524289025329,
 0.2064691774753469,
 0.18232155686728796,
 2.0407327118091434,
 1.83258146375431,
 1.3648724114576,
 1.0925199003723958,
 1.1525737271519403,
 0.18996571979664156]